In [ ]:
# 모듈 선언
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import cv2, dlib
import time
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imutils import face_utils
from google.colab.patches import cv2_imshow
from tqdm import tqdm

# Parameters
TH1 = 0.5  # Confidence
TH2 = 0.25 # Non-maximum Suppression

# CNN model
config = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/yolov3.cfg' 
model = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/yolov3.weights' 
classLabels = '/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/coco.names'

In [ ]:
x_train = np.load('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/dataset/x_train.npy').astype(np.float32)
y_train = np.load('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/dataset/y_train.npy').astype(np.float32)
x_val = np.load('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/dataset/x_val.npy').astype(np.float32)
y_val = np.load('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/dataset/y_val.npy').astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(2586, 26, 34, 1) (2586, 1)
(288, 26, 34, 1) (288, 1)


In [ ]:
# Image Augmentation(옵션을 더 많이 줘서 개선)
train_argmentation = ImageDataGenerator(
    rescale=1./255, # 이미지 0-1 사이의 값으로 변경
    rotation_range=10,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

val_argmentation = ImageDataGenerator(
    rescale=1./255
)

augment_size = 50
train_generator = train_argmentation.flow(
    x=x_train, y=y_train,
    batch_size=augment_size,
    shuffle=True
)
val_generator = val_argmentation.flow(
    x=x_val, y=y_val,
    batch_size=augment_size,
    shuffle=False
)

In [ ]:
# CNN 모델 생성(tensorflow로 개선)
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(input_shape=(26, 34, 1), kernel_size=(3,3), filters=32, strides=1, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=64, padding='same', strides=1, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Conv2D(kernel_size=(3,3), filters=128, padding='same', strides=1, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

cnn_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 34, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 17, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 17, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 8, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 4, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)             

In [ ]:
# 모델 학습
history = cnn_model.fit_generator(train_generator, epochs=25, validation_data=val_generator)

Epoch 1/25


/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


52/52 [==============================] - 3s 36ms/step - loss: 0.4850 - accuracy: 0.7664 - val_loss: 0.2421 - val_accuracy: 0.9097
Epoch 2/25
52/52 [==============================] - 2s 32ms/step - loss: 0.2836 - accuracy: 0.8844 - val_loss: 0.1303 - val_accuracy: 0.9583
Epoch 3/25
52/52 [==============================] - 2s 33ms/step - loss: 0.1822 - accuracy: 0.9308 - val_loss: 0.0776 - val_accuracy: 0.9757
Epoch 4/25
52/52 [==============================] - 2s 32ms/step - loss: 0.1680 - accuracy: 0.9370 - val_loss: 0.0643 - val_accuracy: 0.9757
Epoch 5/25
52/52 [==============================] - 2s 32ms/step - loss: 0.1500 - accuracy: 0.9482 - val_loss: 0.0834 - val_accuracy: 0.9653
Epoch 6/25
52/52 [==============================] - 2s 32ms/step - loss: 0.1192 - accuracy: 0.9571 - val_loss: 0.0591 - val_accuracy: 0.9757
Epoch 7/25
52/52 [==============================] - 2s 31ms/step - loss: 0.1043 - accuracy: 0.9675 - val_loss: 0.0488 - val_accuracy: 0.9826
Epoch 8/25
52/52 [======

In [ ]:
# https://github.com/kairess/eye_blink_detector/blob/master/test.py
IMG_SIZE = (34, 26)

# 얼굴 특징점 추출
detector = dlib.get_frontal_face_detector()  # face detector
predictor = dlib.shape_predictor('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/shape_predictor_68_face_landmarks.dat')

In [ ]:
def crop_eye(img, eye_points):
    x1, y1 = np.amin(eye_points, axis=0)
    x2, y2 = np.amax(eye_points, axis=0)
    cx, cy = (x1 + x2) / 2, (y1 + y2) / 2

    w = (x2 - x1) * 1.2
    h = w * IMG_SIZE[1] / IMG_SIZE[0]

    margin_x, margin_y = w / 2, h / 2

    min_x, min_y = int(cx - margin_x), int(cy - margin_y)
    max_x, max_y = int(cx + margin_x), int(cy + margin_y)

    eye_rect = np.rint([min_x, min_y, max_x, max_y]).astype(np.int)  # 정수 array로 변환

    eye_img = gray[eye_rect[1]:eye_rect[3], eye_rect[0]:eye_rect[2]]

    return eye_img, eye_rect

In [ ]:
# from darknet.py
# yolo
def fun_get_colors(names):
    """
    Create a dict with one random BGR color for each
    class name
    """
    random.seed(1)
    return {name: (
        random.randint(0, 255),
        random.randint(0, 255),
        random.randint(0, 255)) for name in names}

In [ ]:
# main(동영상 저장과 경고 알림 추가)
if __name__ == "__main__":
    # Load CNN model trained over MS COCO DB
    net = cv2.dnn.readNetFromDarknet(config, model)
    layerNames = net.getLayerNames()
    layerOutputs = [layerNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    with open(classLabels, 'rt') as file:
        classNames = file.read().strip('\n').split('\n')
    classColors = fun_get_colors(classNames)
    
    cap = cv2.VideoCapture('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/videos/use_cellphone3.mp4')
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cnt = 0

    for now_frame in tqdm(range(total_frame)):
        ret, img_ori = cap.read()

        if not ret:
            break

        img_ori = cv2.resize(img_ori, dsize=(0, 0), fx=0.5, fy=0.5)

        img = img_ori.copy()
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Run CNN: input image to output tensor
        try:
            blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), swapRB=1)
            # blob = cv2.dnn.blobFromImage(img, 1/255, (320, 320), swapRB=1)
            net.setInput(blob)
            outs = net.forward(layerOutputs)
        except Exception as e:
            print(str(e))

        faces = detector(gray)

        # Non-maximum Suppression
        imgH, imgW, _ = img.shape
        classIDs = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = list(detection[5:])
                confidence = max(scores)
                classID = scores.index(confidence)
                if confidence > TH1:
                    centerX, centerY   = int(detection[0] * imgW), int(detection[1] * imgH)
                    boxW, boxH         = int(detection[2] * imgW), int(detection[3] * imgH)
                    topleftX, topleftY = int(centerX - boxW / 2), int(centerY - boxH / 2)
                    boxes.append([topleftX, topleftY, boxW, boxH])
                    confidences.append(float(confidence))
                    classIDs.append(int(classID))
        resultDet = cv2.dnn.NMSBoxes(boxes, confidences, TH1, TH2)

        # Visualization
        for ind in resultDet:
            id = ind[0]
            topleftX, topleftY, boxW, boxH = boxes[id]
            label = f'{classNames[classIDs[id]]}: {confidences[id]:.2}'
            if "cell phone" in label:
                cv2.putText(img, "Warning!", (100, 300), cv2.FONT_HERSHEY_PLAIN, 12.0, (0, 0, 255), 5); # 입력 이미지, 입력 문구, 문구 시작 위치, 폰트, 글자크기, 글자색상, 글자굵기
            color = classColors[classNames[classIDs[id]]]
            cv2.rectangle(img, (topleftX, topleftY, boxW, boxH), color, 2)
            cv2.putText(img, label, (topleftX, topleftY-12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        eTime, _ = net.getPerfProfile()
        # print("Elapsed time: %.2f ms" % (eTime * 1000.0 / cv.getTickFrequency()))
        
        for face in faces:
            shapes = predictor(gray, face)
            shapes = face_utils.shape_to_np(shapes)

            eye_img_l, eye_rect_l = crop_eye(gray, eye_points=shapes[36:42])
            eye_img_r, eye_rect_r = crop_eye(gray, eye_points=shapes[42:48])

            eye_img_l = cv2.resize(eye_img_l, dsize=IMG_SIZE)
            eye_img_r = cv2.resize(eye_img_r, dsize=IMG_SIZE)
            eye_img_r = cv2.flip(eye_img_r, flipCode=1)

            eye_input_l = eye_img_l.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.
            eye_input_r = eye_img_r.copy().reshape((1, IMG_SIZE[1], IMG_SIZE[0], 1)).astype(np.float32) / 255.

            pred_l = cnn_model.predict(eye_input_l)
            pred_r = cnn_model.predict(eye_input_r)

            # visualize
            state_l = '%.1f' if pred_l > 0.1 else '%.1f'
            state_r = '%.1f' if pred_r > 0.1 else '%.1f'

            state_l = state_l % pred_l
            state_r = state_r % pred_r

            if state_l == '0.0' and state_r == '0.0':
                cnt += 1
            if cnt > 30:
                cv2.putText(img, "Warning!", (100, 300), cv2.FONT_HERSHEY_PLAIN, 12.0, (0, 0, 255), 5)
            if state_l > '0.3' or state_r > '0.3':
                cnt = 0

            cv2.rectangle(img, pt1=tuple(eye_rect_l[0:2]), pt2=tuple(eye_rect_l[2:4]), color=(255,255,255), thickness=2)
            cv2.rectangle(img, pt1=tuple(eye_rect_r[0:2]), pt2=tuple(eye_rect_r[2:4]), color=(255,255,255), thickness=2)

            cv2.putText(img, state_l, tuple(eye_rect_l[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
            cv2.putText(img, state_r, tuple(eye_rect_r[0:2]), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)

        cv2.imwrite(f"/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/{now_frame:.0f}.jpg", img)

100%|██████████| 320/320 [12:01<00:00,  2.25s/it]


In [ ]:
# picture to video
# 참고 : https://hanryang1125.tistory.com/19
import glob

img_array = []

for file in tqdm(glob.glob('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/*.jpg')):
    img = cv2.imread(file)
    img_array.append(img)

height, width = img.shape[:2]
size = (width, height)
video = cv2.VideoWriter('/content/drive/MyDrive/Colab Notebooks/ComputerVision/project/result/result.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)

for img in img_array:
    video.write(img)

video.release()

100%|██████████| 320/320 [00:04<00:00, 78.32it/s]
